<a href="https://colab.research.google.com/github/trachtok/dspracticum2020_data/blob/main/assignment05/assignmen05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning for classification
*Kája Trachtová, Michaela Kecskésová, Martin Špilar, Dagmar Al Tukmachi*

+ goal of this assignment is to take already trained classification model (MobileNetV2) and use it to classify our images
+ there will be 2 classes: Superman vs Batman images

### Workflow of this notebook
1. Load libraries
2. Prepare input data (split images into training and validation set)
3. Download MobileNetV2 model
4. Transfer learning - train MobileNetV2 for our classification
5. Export model to TSjs

## Load libraries

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from google.colab import drive

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
!pip install split-folders
import splitfolders

## Prepare input data

+ Mount Google Discs

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


+ check what data we have, how many images in each class?
+ why there is not the same number of images in each class? **because during model training, accuracy was not high enough so we went through misclassified images and tried to manually delete the most obvious ones (like images where both Batman and Superman were present) -> this helped to increase accuracy**

In [28]:
!ls '/content/drive/My Drive/assignment05_data/batman' | wc -l
!ls '/content/drive/My Drive/assignment05_data/superman' | wc -l

256
289


+ for effective usage of `ImageDataGenerator()` it would be better to have 2 folders: training and validation and in both a specific subset of images for classes batman & superman
+ to obtain such order, we can use package `splitfolders`

In [29]:
splitfolders.ratio("/content/drive/My Drive/assignment05_data", output="/content/drive/My Drive/assignment05_data/prepared_data", seed=1337, ratio=(.8, .2)) 

Copying files: 549 files [00:08, 63.46 files/s]


In [31]:
!ls "/content/drive/My Drive/assignment05_data/prepared_data/train"
!ls "/content/drive/My Drive/assignment05_data/prepared_data/val"

batman	superman
batman	superman


In [32]:
train_dir = "/content/drive/My Drive/assignment05_data/prepared_data/train"
validation_dir = "/content/drive/My Drive/assignment05_data/prepared_data/val"

# Download MobileNetV2
+ download MobileNetV2 model without the top layer

In [33]:
BATCH_SIZE = 32
EPOCHS = 5
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)

In [34]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
# base_model.summary()

# Transfer learning
+ using already trained model, first freeze all layers

In [35]:
for layer in base_model.layers:
    layer.trainable = False

+ then add pooling layer and dropout, finally put on top `softmax` for classification

In [36]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=2, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

+ create `ImageDataGenerator()` instance separately for training a validation set
+ for training dataset, we can set several data augmentation parameters such as rotation, horizontal flip etc. -> it is not generally advised to do the same also for validation dataset, therefore, for validation only neccessary rescaling is set
+ we tried to use several data augmentation parameters, but it only worsened the accuracy of prediction, this might be due to the fact that we do not have enough images to begin with and transforming them only confuses the model?

In [37]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

+ create iterator through training and validation dataset with `.flow_from_directory()`

In [38]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = True)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', 
    shuffle=True)


Found 435 images belonging to 2 classes.
Found 110 images belonging to 2 classes.


In [39]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator)

Epoch 1/5
 7/14 [==============>...............] - ETA: 6s - loss: 0.9166 - accuracy: 0.5580

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


14/14 [==============================] - 19s 1s/step - loss: 0.8301 - accuracy: 0.5862 - val_loss: 0.5701 - val_accuracy: 0.7000
Epoch 2/5
14/14 [==============================] - 18s 1s/step - loss: 0.4638 - accuracy: 0.7655 - val_loss: 0.4272 - val_accuracy: 0.8182
Epoch 3/5
14/14 [==============================] - 18s 1s/step - loss: 0.3213 - accuracy: 0.8644 - val_loss: 0.3805 - val_accuracy: 0.8455
Epoch 4/5
14/14 [==============================] - 18s 1s/step - loss: 0.2753 - accuracy: 0.8805 - val_loss: 0.3603 - val_accuracy: 0.8455
Epoch 5/5
14/14 [==============================] - 18s 1s/step - loss: 0.2432 - accuracy: 0.9011 - val_loss: 0.3398 - val_accuracy: 0.8727


# Export to TSjs

In [40]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 3.2MB/s 
     |████████████████████████████████| 112kB 13.3MB/s 
  Found existing installation: tensorflow-hub 0.10.0
    Uninstalling tensorflow-hub-0.10.0:
      Successfully uninstalled tensorflow-hub-0.10.0


In [127]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "/content/drive/My Drive/assignment05_data/export_model")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [113]:
import shutil
shutil.make_archive("export_model", 'zip', "/content/drive/My Drive/assignment05_data/export_model/", )

from google.colab import files
files.download('export_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>